# Implementação MLP




Funções de ativação

In [50]:
import numpy as np
import pandas as pd
import math
from copy import deepcopy

def sigmoid(x, derivate=False):
    if(derivate):
        # s = sigmoid(x)
        return x * (1 - x)#s * (1 - s)}
    return 1/(1+np.exp(-x))

def gaussian(x, derivate=False):
    if(derivate):
        return -2 * np.exp(-np.power(x,2))
    return np.exp(-np.power(x,2))

def identity(x, derivate=False):
    if(derivate):
        return 1
    return x

def sinusoid(x, derivate=False):
    if(derivate):
        return np.cos(x)
    return np.sin(x)

def relu(x, derivate=False):
    if(derivate):
        x[x >= 0] = 1
        x[not x0s] = 0
    x[x<0] = 0.0
    return x

def softmax(x, derivate=False):
    return np.exp(x) / sum(np.exp(x))


def mean_square_error(expected_data, predicted_data):
    return ((expected_data - predicted_data) ** 2).mean(axis=None)

In [51]:
# K fold

def kFold(data_len, foldsNum):
    indices = [ x[0] for x in np.ndindex(data_len)]
    np.random.shuffle(indices)
    
    numPerFold = int(data_len / foldsNum)
    folds = []
    fold = []
    count = 0
    for i in indices:
        count+=1
        fold.append(i)
        if(count == numPerFold):
            folds.append(fold)
            fold = []
            count = 0
    seq = []
    
    for i in range(foldsNum):
        d = [x for x in range(foldsNum)]
        d.remove(i)
        seq.append((d, i))
    return seq, folds

## Implementação da Rede + Backpropagation


In [52]:
import pdb

class MLPNeuralNetwork():
    def __init__(self, inputNum, hidden):
        self.weights, self.bias, self.activations = self.__create_network__(inputNum, hidden)
        
        
    def __create_network__(self, inputNum, architecture):
        activations = []
        bias = []
        weights = []
        
        layer = architecture[0]
        activations.append(layer[1])
        weights.append(np.random.rand(inputNum, layer[0]))
        bias.append(np.random.rand(layer[0]))
        
        for i in range(1, len(architecture)):
            layer = architecture[i]
            activations.append(layer[1])
            weights.append(np.random.rand(architecture[i-1][0], layer[0]))
            bias.append(np.random.rand(layer[0]))
            
        return weights, bias, activations


    def predict(self, input_, retInt=False):
        if(retInt):
            outputs = []

        output = None
        for i in range(len(self.weights)):
            output = self.activations[i](input_.dot(self.weights[i]) + self.bias[i])
            input_ = output
            
            if(retInt):
                outputs.append(output)
        if(retInt):
            return output, outputs
        else:
            return output
        
    def score(self, inputs, outputs):
        predictions = []
        #pdb.set_trace()
        for input_ in inputs:
            p = self.predict(input_, retInt=False).tolist()
            if(len(p)==1):
                predictions+= p
            else:
                predictions.append(p)
        # 
        return mean_square_error(outputs, np.array(predictions))
    
    
    # calcula os deltas 
    def back_propagate(self, input_, expected, verbose=False):
        # print("back_propagate")
        output, outputs = self.predict(input_, retInt=True)
        if(verbose):
            print("outputs: ", outputs)
        deltas = []
        for index in reversed(range(len(self.weights))):
            error = None
            if index == len(self.weights) - 1:#camada de saída
                delta = (output - expected) * self.activations[index](output, derivate=True) # correto
                if(verbose):
                    print("delta of output layer", delta)
            else:
                delta = self.weights[index+1].dot(deltas[0].T) * self.activations[index](outputs[index], derivate=True)
                if(verbose):
                    print("output layer before is, ", outputs[index])
                    print("weigts * ", self.weights[index+1])
                    print("input t'", outputs[index])
                    print("delta * ", delta[0].T)
                    print("delta result", delta)
            deltas.insert(0,delta)
        return deltas, output, outputs

    def calculate_delta_w(self, deltas, outputs, learnRate, input_):
        w_deltas = []
        bias_deltas = []
        for index in range(len(self.weights)):
            if(index == 0):
                w_deltas.append((-learnRate * input_.T * (deltas[index].reshape(len(deltas[index]), 1))).T)
            else:
                w_deltas.append(np.array(-learnRate * (outputs[index-1].reshape(
                    len(outputs[index-1]), 1))* deltas[index]))
            bias_deltas.append(-learnRate * 1 * deltas[index])
        return w_deltas, bias_deltas
    
    
    def __update_weights_batelada__(self, wDeltas, bDeltas, learnRate, N):
        # pdb.set_trace()
        for index in range(len(self.weights)):
            self.weights[index] +=  wDeltas[index]
            self.bias[index] +=     bDeltas[index]
    
    def update_weights_padrao(self, deltas, outputs, learnRate, input_, verbose=False):
        for index in range(len(self.weights)):
            if(index == 0):
                self.weights[index] += (-learnRate * input_.T * (deltas[index].reshape(len(deltas[index]), 1))).T
            else:
                #pdb.set_trace()
                self.weights[index] += np.array(-learnRate * (outputs[index-1].reshape(
                    len(outputs[index-1]), 1))* deltas[index])
            self.bias[index] +=   -learnRate * 1 * deltas[index]
            
    def __train_network__(self, input_, expected_, learnRate, maxError, num_it):
        for iteration in range(num_it):
            deltas, output, outputs = self.back_propagate( input_, expected_)
            self.update_weights(deltas, outputs, learnRate)
            
            totalError = mean_square_error(expected_, output)
            print("iteration {} error {}".format(iteration, totalError))
            if(totalError < maxError):
                print("Treinamento finalizado")
                break
    
    def train_network_padrao(self, input_, expected_, learnRate, maxError, num_it, randomize=True):
        for iteration in range(num_it):
            totalError = []
            indices = [ x[0] for x in np.ndindex(len(inputs))]

            if randomize:
                np.random.shuffle(indices)

            for index in indices:#np.random.randint(len(input_), size=len(input_)):
                deltas, output, outputs = self.back_propagate( input_[index], expected_[index])
                self.update_weights_padrao(deltas, outputs, learnRate, input_[index])
                #totalError.append(mean_square_error(expected_[index], output))
            #totalError = sum(totalError) / len(totalError)
            totalError = self.score(input_, expected_)
            if(iteration % 1000 == 0):
                print("iteration {} error {}".format(iteration, totalError))
            if(totalError < maxError):
                print("Treinamento finalizado")
                break
        print("num iterações: ", iteration)
                
    def train_network_batelada(self, input_, expected_, learnRate=0.1, alpha=2, maxError=0.01, num_it=10000,
                              randomize=True):
        for iteration in range(num_it):
            wDeltaTotal = None
            bDeltaTotal = None
            totalError = []
            indices = [ x[0] for x in np.ndindex(len(inputs))]

            if randomize:
                np.random.shuffle(indices)

            for index in indices:#np.random.randint(len(input_), size=len(input_)):
                deltas, output, outputs = self.back_propagate( input_[index], expected_[index])
                w_deltas, bDeltas = self.calculate_delta_w(deltas, outputs, learnRate, input_[index])
                # pdb.set_trace()
                
                if(wDeltaTotal is None):
                    wDeltaTotal=w_deltas
                    bDeltaTotal=bDeltas
                else:
                    for i in range(len(wDeltaTotal)):
                        wDeltaTotal[i]+=np.array(w_deltas[i])
                        bDeltaTotal[i]+=np.array(bDeltas[i])
                # print("nxx delta:", wDeltaTotal, "bias", bDeltaTotal)

            self.__update_weights_batelada__(wDeltaTotal, bDeltaTotal, learnRate, len(input_))

            totalError = self.score(input_, expected_)
            
            if(iteration % 1000 == 0):
                print("iteration {} error {}".format(iteration, totalError))
            if(totalError < maxError):
                print("Treinamento finalizado")
                break
        print("num iterações: ", iteration)
        
    def copy(self):
        return deepcopy(self)


# Tarefa 01. XOR

Classificação XOR: (rede 2 => 2 => 1) para avaliar a implementacao do
algoritmo Backpropagation para MLP
Testar com o treinamento por padrão e por batelada

#### Leitura dos dados de treinamento

In [9]:
from numpy import genfromtxt

# read input data
xor_data = genfromtxt('datasets/xor.csv', delimiter=',', skip_header=1)

inputs = xor_data[:,0:2]
outputs = xor_data[:,2]

In [10]:
xor_network = MLPNeuralNetwork(inputNum=2, hidden=[
    (2, sigmoid), (1, sigmoid)])

print("treinamento por padrão")
xor_network.train_network_padrao(inputs, outputs, learnRate=0.3, maxError=0.01, num_it=10000)

print("score padrão: ", xor_network.score(inputs, outputs))

treinamento por padrão
iteration 0 error 0.2626618087012363
iteration 1000 error 0.19069433659569066
iteration 2000 error 0.01168895090784159
Treinamento finalizado
num iterações:  2088
score padrão:  0.009996056619279374


In [11]:
for input_ in inputs:
    print(input_, xor_network.predict(input_))

[0. 0.] [0.09773129]
[0. 1.] [0.90332645]
[1. 0.] [0.90311013]
[1. 1.] [0.10816376]


In [7]:
xor_network = MLPNeuralNetwork(inputNum=2, hidden=[
    (2, sigmoid), (1, sigmoid)])

print("treinamento por batelada")
xor_network.train_network_batelada(inputs, outputs, learnRate=2, maxError=0.01, num_it=10000)

print("score padrão: ", xor_network.score(inputs, outputs))

treinamento por batelada
iteration 0 error 0.2842019664380314
Treinamento finalizado
num iterações:  318
score padrão:  0.009918156249898651


# Iris Dataset Classification

InfoDadosIris.pdf (rede 4 => ? => 1) Versus rede 4 => ? => 3)

- Testar RNs com diferentes números de neurônios na camada de saída:
     - 1 único neurônio
     - 3 neurônios de saída (1 para cada classe)
- Testar diferentes números de neurônios na camada escondida
- Testar diferentes funções de ativação (sigmoide, tg hip, ....)
- Testar diferentes taxas de apresendizado
- Testar Bakpropagation com e sem momento

Para cada caso acima, implementar o 5-fold cross validation e apresentar para
cada caso a média dos resultados de acurácia da rede. As comparações entre os
diferentes parâmetros devem ser feitas com relação à acurácia média em cada
caso.

Obs: verificar a questão da inicialização dos pesos (como isso afeta o
algoritmo)


In [39]:
# read input data
iris_data =  pd.read_csv('datasets/IrisDataset.csv', delimiter=',')

convOne = {'Iris-setosa': 0,
'Iris-versicolor': 1,
 'Iris-virginica': 2
}

convThree = {'Iris-setosa': np.array([1, 0, 0]),
'Iris-versicolor': np.array([0, 1, 0]),
 'Iris-virginica': np.array([0, 0, 1])
}

In [40]:
inputs  = np.array(iris_data.iloc[:,0:4])
outputs = np.array(iris_data.iloc[:,-1])


outputsOne = np.array([convOne[x] for x in outputs] )

outputsThree = np.array([convThree[x] for x in outputs]) 

In [68]:
def crossMLP(network_base, inputs, outputs, foldsNum=5):
    seq, folds = kFold(len(inputs), foldsNum)
    
    score = []
    for index in range(foldsNum):
        network = network_base.copy()
        _train, _test = seq[index]
        train = []
        for x in _train:
            train += folds[x]
        test =  folds[_test]
        print(train)
        network.train_network_padrao(inputs[train], outputs[train], learnRate=0.1, 
                                 maxError=0.05, num_it=10000)
        score.append(network.score(inputs[test], outputs[test]))
    
    return sum(score)/foldsNum
        

## 1 neurônio na camada de saída

In [ ]:
iris_network = MLPNeuralNetwork(inputNum=4, hidden=[
(10, sigmoid), (1, sigmoid)])

iris_network.train_network_padrao(inputs, outputsOne, learnRate=0.1, 
                                 maxError=0.05, num_it=10000)

iris_network.score(inputs, outputOne)

iteration 0 error 0.6665392595404562
iteration 1000 error 0.33349850045325086
iteration 2000 error 0.3334133711388228
iteration 3000 error 0.33338611063666046
iteration 4000 error 0.3333726908630321
iteration 5000 error 0.3333647076394897
iteration 6000 error 0.33335941468013774
iteration 7000 error 0.3333556485795166
iteration 8000 error 0.3333528321329673


In [ ]:
score = crossMLP(iris_network, inputs, outputsOne)

## 3 neurônios na camada de saída

In [28]:
iris_network = MLPNeuralNetwork(inputNum=4, hidden=[
(20, sigmoid), (3, sigmoid)])

iris_network.train_network_padrao(inputs, outputsThree, learnRate=0.1, 
                                 maxError=0.05, num_it=10000)

iris_network.score(inputs, outputsThree)

iteration 0 error 0.666623108571756
Treinamento finalizado
num iterações:  547


0.03779873983210967

In [32]:
iris_network = MLPNeuralNetwork(inputNum=4, hidden=[
(20, sigmoid), (3, sigmoid)])

iris_network.train_network_batelada(inputs, outputsThree, learnRate=0.1, 
                                 maxError=0.05, num_it=10000)

iris_network.score(inputs, outputsThree)

iteration 0 error 0.6666428939294694
iteration 1000 error 0.3333331502998912
iteration 2000 error 0.33333313426503675
iteration 3000 error 0.33333311514583674
iteration 4000 error 0.33333309195820715
iteration 5000 error 0.33333306324975903
iteration 6000 error 0.33333302678368726
iteration 7000 error 0.3333329789267003
iteration 8000 error 0.33333291335801063
iteration 9000 error 0.3333328180216999
num iterações:  9999


0.33333266691263763

# Facebook Metrics Regression

 Regressão: Dados Facebook Metrics (rede 18 => ? => 1)


In [ ]:
facebook_data = pd.read_csv('datasets/dataset_Facebook.csv', delimiter=';')

#print(facebook_data.head())
